In [ ]:
import rqdatac, pandas, datetime

rqdatac.init()
index_components = rqdatac.index_components('000300.XSHG', date=datetime.date(2025, 11, 6))

data_list = []
for symbol in index_components: # pyright: ignore[reportOptionalIterable]
    df = rqdatac.get_price(symbol, start_date=20251001, end_date="2025-11-01")
    data_list.append(df)
all_data = pandas.concat(data_list)
all_data.to_parquet("test_data.parquet")

In [1]:
import pandas as pd
stock_data = pd.read_parquet("test_data.parquet")
stock_data = stock_data.reset_index()
factor_data = stock_data[["order_book_id", "date"]].copy()
factor_data["factor_value"] = stock_data.groupby("order_book_id")["close"].rolling(5).mean().values
factor_data.fillna(0.0,inplace=True)

In [3]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"项目根目录已添加至 sys.path: {project_root}")

from longgang_trader.backtesting.backtester import Backtester, BaseStrategy

# 定义一个简单的分层回测策略
class SimpleLayeredStrategy(BaseStrategy):
    """一个简单的策略，在每个交易日，等权重买入因子值最高的一只股票"""
    def generate_signals_for_all_dates(self):
        # 合并因子数据，方便处理
        data = self.factor_data.copy()
        
        # 在每个日期，找到因子值最高的股票
        top_stocks = data.loc[data.groupby(self.date_col)[self.factor_value_col].idxmax()]
        
        # 设置目标权重为1.0
        top_stocks[self.weight_col] = 1.0

        signals = top_stocks[[self.date_col, self.symbol_col, self.weight_col]]
        #signals.columns = ['date', 'symbol', 'target_weight']
        
        print("\n生成的交易信号 (signals):")
        print(signals)
        return signals
    
strategy_config = {
    "date_col": "date",
    "symbol_col": "order_book_id",
    "factor_value_col": "factor_value",
    "weight_col": "target_weight"
}


# --- 运行回测 ---
print("\n--- 开始执行回测 ---")
# 1. 初始化策略
strategy = SimpleLayeredStrategy(factor_data, strategy_config)

项目根目录已添加至 sys.path: d:\Notes\Project\LongGangTrader

--- 开始执行回测 ---


In [4]:

# 2. 初始化回测器
# 注意：确保你已经成功运行了 `maturin develop`
config_dict = {
    "initial_capital": 10_000_000.0,
    "transaction_cost": 0.001,
    "symbol_col": "order_book_id",
    "date_col": "date",
    "close_col": "close"
}
bt = Backtester(
    strategy=strategy,
    data=stock_data,
    config=config_dict
)

# 3. 运行回测（这将调用 Rust 核心）
bt.run_backtest()
portfolio_history = bt.get_portfolio_history()


生成的交易信号 (signals):
           date order_book_id  target_weight
0    2025-10-09   002714.XSHE            1.0
1    2025-10-10   002714.XSHE            1.0
2    2025-10-13   002714.XSHE            1.0
3    2025-10-14   002714.XSHE            1.0
2724 2025-10-15   600690.XSHG            1.0
2725 2025-10-16   600690.XSHG            1.0
2726 2025-10-17   600690.XSHG            1.0
2727 2025-10-20   600690.XSHG            1.0
2728 2025-10-21   600690.XSHG            1.0
2729 2025-10-22   600690.XSHG            1.0
2730 2025-10-23   600690.XSHG            1.0
2731 2025-10-24   600690.XSHG            1.0
2732 2025-10-27   600690.XSHG            1.0
4977 2025-10-28   000725.XSHE            1.0
4978 2025-10-29   000725.XSHE            1.0
4979 2025-10-30   000725.XSHE            1.0
4980 2025-10-31   000725.XSHE            1.0
Rust 回测成功完成。


In [5]:
portfolio_history

,date,equity,cash,holdings_value
0,2025-10-09,1.000000e+07,-10000.000000,-0.000000e+00
1,2025-10-10,1.006001e+07,-10.000000,1.007001e+07
2,2025-10-13,1.003732e+07,-0.010000,1.003733e+07
3,2025-10-14,1.015073e+07,-0.000010,1.015073e+07
4,2025-10-15,1.006945e+07,-20138.901524,1.006945e+07
5,2025-10-16,9.985907e+06,-20.138902,1.000605e+07
6,2025-10-17,9.894926e+06,-0.020139,9.894946e+06
7,2025-10-20,9.922610e+06,-0.000020,9.922610e+06
8,2025-10-21,1.002543e+07,-0.000020,1.002543e+07
9,2025-10-22,1.012826e+07,-0.000020,1.012826e+07
